In [16]:
import numpy as np
def v(x, y, z=0):
    return np.array([x, y, z])

import tempfile
from pathlib import Path

from solid import *
from solid.utils import *
import viewscad

from subprocess import run


R=viewscad.Renderer()

wall_size=2
fanZ=15
fanX=58
fanY=55
fan=v(fanX, fanY, fanZ)

bedZ=17
bedX=50
bedY=50
bed=v(bedX, bedY, bedZ)

def XYZ(X=0, Y=0, Z=0):
    return lambda *args : translate(v(X, Y, Z))(*args)

def NULLOBJ():
    return cube(0.1)

debug     = False

def dprint(msg):
    if debug:
        print(msg)

def SCADdir():
    return Path("scad")

def STLdir():
    return Path("stl")

def render_to_stl(name, object):
    scad_dir=SCADdir()
    scad_dir.mkdir(parents=True, exist_ok=True)
    dprint(f'Making scad files in {scad_dir.as_posix()}')
    stl_dir = STLdir()
    stl_dir.mkdir(parents=True, exist_ok=True)
    stl_file_name = stl_dir / Path(name+".stl")
    scad_file_name = scad_dir / Path(name+".scad")
    dprint(f'stl file name == {stl_file_name.as_posix()}')
    dprint(f'scad file name == {scad_file_name.as_posix()}')
    # generate valid openscad code and store it in file
    scad_render_to_file(object, scad_file_name)
    # run openscad and export to stl
    run(["openscad", "--render", "-o",  
         stl_file_name.as_posix(), 
         scad_file_name.as_posix()])

def make_part(name, object):
    render_to_stl(name, object)
    return object

def remove_dir(dirpath):
    if dirpath.exists():
        if not dirpath.is_dir():
            dirpath.unlink()
        else:
            files = []
            for child in dirpath.iterdir(): 
                files.append(child)
            for child in files:
                if child.is_dir():
                    remove_dir(child)
                else:
                    child.unlink()
            dirpath.rmdir()

def cleanup():
    remove_dir(SCADdir())
    remove_dir(STLdir())

In [35]:
@bom_part("Clamp Jaw")
def clamp_jaw():
    part = cube(jaw) - 
    return make_part("clamp_jaw", part)

@bom_part("Clamp Screw")
def clamp_screw():
    part = NULLOBJ()
    return make_part("clamp_screw", part)      

@bom_part("Fan Control Box")
def fanCTL(rotation=v(0, 0, 0), offset=v(0, 0, 0)):
    part = translate(offset) (
               rotate(rotation) (
                   cube(fan + v(wall_size, 0, wall_size), center=False)
               )
            )
    return make_part("fanctl", part)

def bedCTL(rotation=v(0, 0, 0), offset=v(0, 0, 0)):
    outer = cube(bed)
    inner = XYZ(wall_size, 2*wall_size, wall_size)(
                cube(bed - v(2*wall_size, 0, 2*wall_size)))
    part = translate(offset) (
               rotate(rotation)(
                   outer - inner
               )
            )
    return make_part("bedctl", part)

@bom_part("Base and Hook")
def base_and_hook():
    part = NULLOBJ()
    return make_part("base_and_hook", part)

def make_all_parts():
    cleanup()
    base_and_hook()
    fanCTL()
    bedCTL()
    clamp_screw()
    clamp_screw()
    clamp_jaw()
    clamp_jaw()


In [37]:
R.render(fanCTL())

Geometries in cache: 3
Geometry cache size in bytes: 2184
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.000
   Top level object is a 3D object:
   Facets:          6
Geometries in cache: 3
Geometry cache size in bytes: 2184
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.000
   Top level object is a 3D object:
   Facets:          6
/home/poppa/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
